In [1]:
%load_ext autoreload
%autoreload 2

if __name__ == '__main__' and __package__ is None:
    from os import sys, path
    sys.path.append(path.dirname(path.dirname(path.abspath("__file__"))))

In [2]:
import pandas as pd
import numpy as np

SEED = 43

In [3]:
df_raw = pd.read_csv('../data/raw/train_data.csv')
df_raw.head()

,index,subject,timestamp,x,y,target
0,0,Subject_1,1.571841e+09,2085,1259,0
1,1,Subject_1,1.571841e+09,2085,1258,0
2,2,Subject_1,1.571841e+09,2087,1258,0
3,3,Subject_1,1.571841e+09,2088,1258,0
4,4,Subject_1,1.571841e+09,2090,1258,0


In [4]:
df_raw.groupby(['target']).index.count()

target
0    830951
1    268071
2     13001
3    226980
Name: index, dtype: int64

In [5]:
df_sub_1=df_raw[df_raw['subject']=='Subject_1'].reset_index()
df_sub_1['timestamp']= df_sub_1['timestamp'] - df_sub_1.loc[0,'timestamp']
df_sub_1.head()

,level_0,index,subject,timestamp,x,y,target
0,0,0,Subject_1,0.000000,2085,1259,0
1,1,1,Subject_1,0.001261,2085,1258,0
2,2,2,Subject_1,0.001933,2087,1258,0
3,3,3,Subject_1,0.002598,2088,1258,0
4,4,4,Subject_1,0.003226,2090,1258,0


### Testing Features

In [6]:
from src.features import main_preprocess

In [7]:
subject=df_sub_1.loc[0,'subject']

In [8]:
main_preprocess.process_files(df_sub_1,subject,1)

,traveled_distance_pixel,elapsed_time,direction_of_movement,straightness,num_points,sum_of_angles,mean_curv,sd_curv,max_curv,min_curv,...,max_a,min_a,mean_jerk,sd_jerk,max_jerk,min_jerk,a_beg_time,n_from,n_to,target
0,229.469000,1.719365,6,0.794105,159,-154.941248,-0.044122,0.912361,1.570796,-1.570796,...,3.269637e+07,-7.154328e+06,1.006021e+09,1.977163e+10,1.171528e+11,-1.679307e+11,0.001933,2,1,0
1,808.418722,1.095563,6,0.875548,410,-495.756708,-0.010170,0.672391,3.141593,-1.570796,...,2.803281e+07,-7.299863e+06,4.234015e+08,6.986681e+09,1.115209e+11,-4.420644e+10,0.000941,2,1,0
2,156.651570,1.192134,2,0.676781,113,169.126857,0.060723,1.002446,1.570796,-1.570796,...,2.642261e+06,-2.252796e+06,4.945293e+07,1.186972e+09,2.931216e+09,-4.374432e+09,0.001483,2,1,0
3,747.010254,1.237152,2,0.593599,251,474.611624,0.029552,0.509560,3.141593,-1.570796,...,3.456051e+07,-1.735078e+07,2.199592e+08,1.212531e+10,9.902172e+10,-1.075885e+11,0.001710,2,1,0
4,1201.086385,1.540254,7,0.890828,458,-124.887349,0.013982,0.591670,4.712389,-4.712389,...,4.344385e+07,-1.033933e+07,3.755695e+08,9.125044e+09,1.625407e+11,-6.410429e+10,0.004076,2,1,0
5,91.242641,2.633629,7,0.629687,96,-25.918139,-0.011539,1.089906,4.712389,-3.141593,...,1.637418e+06,-2.838246e+05,7.377859e+07,3.954050e+08,2.104078e+09,-9.483841e+08,0.000000,2,1,0
6,641.784131,1.195047,3,0.928082,242,680.461767,0.038232,0.581927,3.141593,-3.141593,...,2.770275e+07,-1.277563e+07,3.911352e+08,7.706183e+09,6.979708e+10,-4.274387e+10,0.000000,2,1,0
7,325.820374,1.666126,2,0.796497,258,448.784102,0.002777,0.924763,1.570796,-1.570796,...,4.630133e+06,-2.341074e+06,1.776823e+08,1.340138e+09,7.010674e+09,-5.202610e+09,0.002869,2,1,0
8,4.000000,2.825653,2,1.000000,15,6.283185,1.570796,0.000000,1.570796,1.570796,...,1.390439e+06,-5.913898e+05,6.665355e+07,5.340755e+08,1.639563e+09,-9.939489e+08,0.000000,2,1,0
9,14.414214,1.136152,1,0.858133,16,18.064158,0.128631,0.917807,1.570796,-1.570796,...,1.080331e+06,-2.508941e+05,1.929914e+07,5.650106e+08,1.165597e+09,-1.329226e+09,0.003413,2,1,0


#### Pegando o Dataset completo

In [9]:
df_subjects = []
for subject in df_raw['subject'].unique():
    df_sub=df_raw[df_raw['subject']==subject].reset_index()
    df_sub['timestamp']= df_sub['timestamp'] - df_sub.loc[0,'timestamp']
    df_subjects.append(main_preprocess.process_files(df_sub,subject,1))

In [10]:
len(df_subjects)

4

In [11]:
df_subjects=pd.concat(df_subjects)
df_subjects.head()

,traveled_distance_pixel,elapsed_time,direction_of_movement,straightness,num_points,sum_of_angles,mean_curv,sd_curv,max_curv,min_curv,...,max_a,min_a,mean_jerk,sd_jerk,max_jerk,min_jerk,a_beg_time,n_from,n_to,target
0,229.469000,1.719365,6,0.794105,159,-154.941248,-0.044122,0.912361,1.570796,-1.570796,...,3.269637e+07,-7.154328e+06,1.006021e+09,1.977163e+10,1.171528e+11,-1.679307e+11,0.001933,2,1,0
1,808.418722,1.095563,6,0.875548,410,-495.756708,-0.010170,0.672391,3.141593,-1.570796,...,2.803281e+07,-7.299863e+06,4.234015e+08,6.986681e+09,1.115209e+11,-4.420644e+10,0.000941,2,1,0
2,156.651570,1.192134,2,0.676781,113,169.126857,0.060723,1.002446,1.570796,-1.570796,...,2.642261e+06,-2.252796e+06,4.945293e+07,1.186972e+09,2.931216e+09,-4.374432e+09,0.001483,2,1,0
3,747.010254,1.237152,2,0.593599,251,474.611624,0.029552,0.509560,3.141593,-1.570796,...,3.456051e+07,-1.735078e+07,2.199592e+08,1.212531e+10,9.902172e+10,-1.075885e+11,0.001710,2,1,0
4,1201.086385,1.540254,7,0.890828,458,-124.887349,0.013982,0.591670,4.712389,-4.712389,...,4.344385e+07,-1.033933e+07,3.755695e+08,9.125044e+09,1.625407e+11,-6.410429e+10,0.004076,2,1,0


In [12]:
df_subjects.target.unique()

array([0, 1, 3, 2])

In [13]:
df_subjects.shape

(13651, 41)

In [14]:
df_subjects.groupby(['target'])['target'].count()

target
0    4086
1    4733
2     206
3    4626
Name: target, dtype: int64

In [15]:
df_subjects.isna().sum()

traveled_distance_pixel    0
elapsed_time               0
direction_of_movement      0
straightness               0
num_points                 0
sum_of_angles              0
mean_curv                  0
sd_curv                    0
max_curv                   0
min_curv                   0
mean_omega                 0
sd_omega                   0
max_omega                  0
min_omega                  0
largest_deviation          0
dist_end_to_end_line       0
num_critical_points        0
mean_vx                    0
sd_vx                      0
max_vx                     0
min_vx                     0
mean_vy                    0
sd_vy                      0
max_vy                     0
min_vy                     0
mean_v                     0
sd_v                       0
max_v                      0
min_v                      0
mean_a                     0
sd_a                       0
max_a                      0
min_a                      0
mean_jerk                  0
sd_jerk       

### Train the model

In [16]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import xgboost as xgb




In [17]:
df = shuffle(df_subjects,random_state=SEED)
df.head()

,traveled_distance_pixel,elapsed_time,direction_of_movement,straightness,num_points,sum_of_angles,mean_curv,sd_curv,max_curv,min_curv,...,max_a,min_a,mean_jerk,sd_jerk,max_jerk,min_jerk,a_beg_time,n_from,n_to,target
2759,362.482681,0.993992,7,0.526929,69,0.639170,-0.080210,1.084860,2.748894,-4.712389,...,7.080518e+05,-1.847267e+05,7.021243e+06,3.916897e+07,1.941283e+08,-8.887111e+07,0.023748,2,1,1
2156,7.000000,4.616354,2,0.868966,8,12.566371,0.448799,0.614543,1.570796,1.570796,...,1.910453e+06,-8.877960e+04,-1.415882e+06,1.651324e+09,2.858193e+09,-2.862140e+09,0.032773,2,1,0
4363,3.828427,34.168284,3,0.941784,4,7.853982,0.817160,0.578256,1.666081,0.785398,...,2.414637e+04,-9.358504e+02,3.361920e+03,3.150109e+06,3.155151e+06,-3.145066e+06,0.007653,2,1,3
1901,143.566291,2.852760,6,0.778540,94,-78.557358,0.013608,1.127878,4.712389,-4.712389,...,1.120095e+07,-6.879614e+06,2.411110e+08,3.626891e+09,1.268224e+10,-1.919397e+10,0.017723,2,1,0
4269,163.941008,1.029697,7,0.938966,36,-12.003044,0.000971,0.418119,1.570796,-1.570796,...,5.986443e+05,-2.563392e+05,1.125559e+07,5.137564e+07,1.664118e+08,-8.303187e+07,0.003637,2,1,1


In [18]:
df_train, df_test = train_test_split(df,test_size=0.2,random_state=SEED)


In [19]:
X_train, y_train = df_train.iloc[:,:-1],df_train.iloc[:,-1]
d_matrix_train = xgb.DMatrix(data=X_train,label=y_train.values)


X_test, y_test = df_test.iloc[:,:-1],df_test.iloc[:,-1]
d_matrix_test = xgb.DMatrix(data=X_test,label=y_test.values)

In [20]:
numofclasses=df.target.nunique()

In [21]:
param = {
    'eta': 0.3, 
    'max_depth': 30,
    'num_class': numofclasses,
    'objective': 'multi:softprob'
    } 

steps = 20  # The number of training iterations

In [22]:
model = xgb.train(param, d_matrix_train, steps)


In [23]:
preds = model.predict(d_matrix_test)
y_pred = np.asarray([np.argmax(line) for line in preds])


In [24]:
y_pred

array([1, 3, 0, ..., 3, 3, 3])

### Evaluate the model

In [25]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [26]:
confusion_matrix(y_test,y_pred)

array([[777,   5,   1,   3],
       [ 13, 889,   0,  56],
       [  1,   1,   1,  40],
       [ 15,  48,   0, 881]])

In [27]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       786
           1       0.94      0.93      0.94       958
           2       0.50      0.02      0.04        43
           3       0.90      0.93      0.92       944

   micro avg       0.93      0.93      0.93      2731
   macro avg       0.83      0.72      0.72      2731
weighted avg       0.93      0.93      0.93      2731



### Saving the model

In [28]:
model.save_model('../models/basic_XGB_model_1.model')